In [ ]:
import os
import cv2
import torch
import matplotlib.pyplot as plt

# Charger le modèle
model_path = r'C:/Users/dell/yolov5/models/best.pt'
model = torch.hub.load(r'C:/Users/dell/yolov5', 'custom', path=model_path, source='local')
model.eval()

# Dossier d’images à analyser
image_folder = r'C:/Users/dell/Downloads/test/images'
output_folder = r'C:/Users/dell/Downloads/test/outputs'  # Dossier où les images annotées seront sauvegardées

# Créer le dossier de sortie s'il n'existe pas
os.makedirs(output_folder, exist_ok=True)

# Récupérer la liste des fichiers images (formats courants)
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Vérifier qu’il y a des images
if not image_files:
    print("Aucune image trouvée dans le dossier.")
else:
    for file in image_files:
        image_path = os.path.join(image_folder, file)
        img = cv2.imread(image_path)

        if img is None:
            print(f"Impossible de lire {file}")
            continue

        # Inference YOLOv5
        results = model(img)

        # Annoter l'image
        annotated_img = results.render()[0]

        # Sauvegarder l'image annotée
        output_image_path = os.path.join(output_folder, file)
        cv2.imwrite(output_image_path, annotated_img)
        print(f"Image sauvegardée : {output_image_path}")


In [3]:
import torch
from pathlib import Path
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Charger le modèle YOLOv5
model_path = r'C:/Users/dell/yolov5/models/best.pt'
model = torch.hub.load(r'C:/Users/dell/yolov5', 'custom', path=model_path, source='local')

# Charger les images et leurs annotations (le dossier "test" par exemple)
img_folder = 'C:/Users/dell/Downloads/test/images'  # Remplace par le chemin de tes images
labels_folder = 'C:/Users/dell/Downloads/test/labels'  # Remplace par le chemin des annotations .txt

# Effectuer les prédictions sur le dossier d'images
results = model(img_folder)

# Accéder aux résultats des prédictions
pred_boxes = results.xywh[0].cpu().numpy()  # Coordonnées des boxes prédictes
pred_classes = results.pred[0][:, -1].cpu().numpy()  # Classes prédites
pred_scores = results.pred[0][:, -2].cpu().numpy()  # Scores de confiance

# Récupérer les classes réelles (en supposant que les annotations sont en format YOLO .txt)
true_classes = []
for label_file in Path(labels_folder).glob('*.txt'):
    with open(label_file, 'r') as f:
        for line in f:
            # Chaque ligne dans les fichiers .txt est dans le format : class x_center y_center width height
            true_classes.append(int(line.split()[0]))  # Classe réelle (ID de la classe)

# Calculer la matrice de confusion
conf_matrix = confusion_matrix(true_classes, pred_classes)

# Afficher la matrice de confusion
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=model.names, yticklabels=model.names)
plt.xlabel('Prédictions')
plt.ylabel('Véritables valeurs')
plt.title('Matrice de confusion')
plt.show()


YOLOv5  v6.0-0-g956be8e6 torch 2.4.1+cpu CPU

C:\Users/dell/yolov5\models\experimental.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_downloa

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\dell\\Downloads\\test\\images'

In [5]:
import torch
import os
from pathlib import Path
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Chemins
model_path = r'C:/Users/dell/yolov5/models/best.pt'
img_folder = Path('C:/Users/dell/Downloads/test/images')
labels_folder = Path('C:/Users/dell/Downloads/test/labels')

# Charger le modèle
model = torch.hub.load(r'C:/Users/dell/yolov5', 'custom', path=model_path, source='local')

# Initialisation
y_true = []
y_pred = []

# Boucle image par image
for img_file in img_folder.glob('*.jpg'):
    # Prédiction
    results = model(str(img_file))
    preds = results.xywh[0].cpu().numpy()

    # Lire les annotations réelles
    label_file = labels_folder / (img_file.stem + '.txt')
    if not label_file.exists():
        continue

    with open(label_file) as f:
        gt_lines = f.readlines()

    # Nombre minimal entre prédictions et vérités terrain
    n = min(len(gt_lines), len(preds))

    for i in range(n):
        gt_class = int(gt_lines[i].split()[0])
        pred_class = int(preds[i][5])
        y_true.append(gt_class)
        y_pred.append(pred_class)


YOLOv5  v6.0-0-g956be8e6 torch 2.4.1+cpu CPU

C:\Users/dell/yolov5\models\experimental.py:94: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_downloa

In [7]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
import matplotlib.pyplot as plt
plt.show()


C:\Users\dell\AppData\Local\Temp\ipykernel_1112\678492309.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [8]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Exemple de noms de classes (à adapter à ton cas)
class_names = ['fall-action', 'sit', 'stand']  # Remplace par tes vraies classes

# Génération de la matrice de confusion
cm = confusion_matrix(y_true, y_pred, labels=range(len(class_names)))

# Affichage avec les noms de classes
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=45)

# Sauvegarde de l'image
plt.title("Matrice de confusion")
plt.tight_layout()
plt.savefig('C:/Users/dell/Downloads/test/outputs/confusion_matrix.png')
print("Matrice de confusion sauvegardée avec noms de classes.")


Matrice de confusion sauvegardée avec noms de classes.
